In [8]:
# Google Colab에서 노트북을 실행하실 때에는
# https://tutorials.pytorch.kr/beginner/colab 를 참고하세요.
%matplotlib inline

In [12]:
#신경망 정의하기

#import
import torch
import torch.nn as nn
import torch.nn.functional as F

#신경망 class 만들기
class Net(nn.Module):

    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = nn.Conv2d(1,6,5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        #아핀 연산 : y = Wx + b
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84,10)

    def forward(self, x):
        #max pooling
        x = F.max_pool2d(F.relu(self.conv1(x)),(2,2))
        # 크기가 제곱수라면, 하나의 숫자만을 특정
        x = F.max_pool2d(F.relu(self.conv2(x)),2)
        x = torch.flatten(x,1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


In [13]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 5, 5])


In [15]:
input = torch.randn(1,1,32,32)
out = net(input)
print(out.item)

RuntimeError: a Tensor with 10 elements cannot be converted to Scalar

In [16]:
net.zero_grad()
out.backward(torch.randn(1,10))

In [23]:
#손실함수

#평균제곱오차 MSELoss
output = net(input)
target = torch.randn(10)
target = target.view(1, -1)
criterion = nn.MSELoss()
loss = criterion(output, target)
print(loss)

tensor(0.9784, grad_fn=<MseLossBackward0>)


In [24]:
loss.grad_fn

In [25]:
print(loss.grad_fn)

In [26]:
print(loss.grad_fn)
print(loss.grad_fn.next_functions[0][0])
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])

In [27]:
#역전파
net.zero_grad()

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()
print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([-0.0076,  0.0066, -0.0101, -0.0125,  0.0139, -0.0072])


In [28]:
#가충치 갱신

#확률적 경사하강법
#weight = weight - learning_rate * gradient

learning_rate = 0.01
for f in net.parameters():
    f.data.sub_(f.grad.data * learning_rate)

In [29]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr = 0.01)

#학습과정
optimizer.zero_grad()
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()